In [1]:
from flask import Flask
from flask import request
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table
import pandas as pd
import numpy as np
import json
import re
import plotly_utilities as pu
from dashapp import dashapp2 as dashapp
import pathlib
import datetime
import re

In [90]:
year = 2021
month = 11
day = 11

ice_brent_txt = open(f'{pathlib.Path.home()}/downloads/B_{year}_{month}_{day}.txt').readlines()
cols = [
    'commod','mmmyy','strike','pc','delta',
    'open','high','low','close','settle','price_change',
    'volume','oi','oi_change','exer','block_volume','eoo','spread_volume'
]
csv_lines = [
#     ','.join(goodline[0].split(' ')) for goodline in 
#     goodline[0].replace(' ',',').split(',') for goodline in 
    goodline[0].split(' ') for goodline in 
    [
        re.findall("B [A-Z][a-z]{2}[2-3][0-9] [1-9][0-9]{1,4}\.[0-9]{2,4} [PC] .+",line)
        for line in ice_brent_txt
    ]
    if len(goodline)>0
]

csv_lines = [
    line if len(line)==18 else line[0:5] + [line[5],line[5],line[5],line[5],line[5]] + line[6:]
    for line in csv_lines
]

df_brent_crude_ice = pd.DataFrame(
    csv_lines,
    columns=cols
)

df_brent_crude_ice = df_brent_crude_ice.fillna(0)
poss_float_cols = [
    'strike','delta',
    'open','high','low','close','settle','price_change',
    'volume','oi','oi_change','exer','block_volume','eoo','spread_volume'
]
float_cols = [c for c in poss_float_cols if c in df_brent_crude_ice.columns.values]
for col in float_cols:
    df_brent_crude_ice[col] = df_brent_crude_ice[col].str.replace(',','').astype(float) 
df_brent_crude_ice

,commod,mmmyy,strike,pc,delta,open,high,low,close,settle,price_change,volume,oi,oi_change,exer,block_volume,eoo,spread_volume
0,B,Jan22,34.00,C,0.9996,48.87,48.87,48.87,48.87,48.87,0.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,B,Jan22,39.00,C,0.9992,43.87,43.87,43.87,43.87,43.87,0.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,B,Jan22,47.00,C,0.9980,35.88,35.88,35.88,35.88,35.88,0.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,B,Jan22,49.00,C,0.9975,33.88,33.88,33.88,33.88,33.88,0.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,B,Jan22,49.75,C,0.9973,33.13,33.13,33.13,33.13,33.13,0.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,B,Dec24,45.00,P,-0.1590,4.74,4.74,4.74,4.74,4.74,-0.27,0.0,750.0,0.0,0.0,0.0,0.0,0.0
1796,B,Dec24,60.00,P,-0.3018,9.79,9.79,9.79,9.79,9.79,-0.32,0.0,1680.0,0.0,0.0,0.0,0.0,0.0
1797,B,Dec25,20.00,P,-0.0299,0.88,0.88,0.88,0.88,0.88,-0.13,0.0,390.0,0.0,0.0,0.0,0.0,0.0
1798,B,Dec25,40.00,P,-0.1377,4.50,4.50,4.50,4.50,4.50,-0.26,0.0,1170.0,0.0,0.0,0.0,0.0,0.0


In [91]:
c_1 = df_brent_crude_ice.mmmyy=='Dec22'
c_2 = df_brent_crude_ice.pc == 'C'
c_3 = df_brent_crude_ice['strike'] >= 100
c_all = c_1 & c_2 & c_3
df_brent_crude_ice[c_all]

,commod,mmmyy,strike,pc,delta,open,high,low,close,settle,price_change,volume,oi,oi_change,exer,block_volume,eoo,spread_volume
831,B,Dec22,100.0,C,0.2342,2.83,2.90,2.83,2.90,2.84,0.07,1500.0,25979.0,468.0,0.0,1000.0,0.0,1000.0
832,B,Dec22,105.0,C,0.1905,2.20,2.20,2.20,2.20,2.20,0.07,0.0,374.0,0.0,0.0,0.0,0.0,0.0
833,B,Dec22,110.0,C,0.1555,1.72,1.72,1.72,1.72,1.72,0.06,40.0,328.0,40.0,0.0,0.0,0.0,40.0
834,B,Dec22,120.0,C,0.1053,1.09,1.09,1.09,1.09,1.09,0.03,40.0,14615.0,40.0,0.0,0.0,0.0,40.0
835,B,Dec22,125.0,C,0.0877,0.89,0.89,0.89,0.89,0.89,0.03,0.0,7220.0,0.0,0.0,0.0,0.0,0.0
836,B,Dec22,130.0,C,0.0738,0.74,0.74,0.74,0.74,0.74,0.03,0.0,500.0,0.0,0.0,0.0,0.0,0.0
837,B,Dec22,131.0,C,0.0714,0.71,0.71,0.71,0.71,0.71,0.02,0.0,500.0,0.0,0.0,0.0,0.0,0.0
838,B,Dec22,140.0,C,0.0535,0.52,0.52,0.52,0.52,0.52,0.01,0.0,250.0,0.0,0.0,0.0,0.0,0.0
839,B,Dec22,150.0,C,0.0399,0.38,0.38,0.38,0.38,0.38,0.00,0.0,2600.0,0.0,0.0,0.0,0.0,0.0
840,B,Dec22,175.0,C,0.0216,0.20,0.20,0.20,0.20,0.20,0.00,0.0,2500.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# server = Flask(__name__)
# app = dash.Dash(
#     __name__,
#     server = server,
#     serve_locally = False,
#     requests_pathname_prefix = "/plotary/dash/",
# )

app = dash.Dash(__name__,url_base_pathname="/plotary/dash/")

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
        html.Div([
#         html.A([
#             html.Div(['goto man-es page'],id="logo")
#         ],href='https://man-es.com'),
        html.Div([
            html.H1('Keine Auswertung ausgewählt. ')
        ],id="description"),
        # This Link does not seem to be causing an invocation of the callback 
        dcc.Link('Navigate to https://man-es.com', href='https://man-es.com'),
        html.Br(),
        # These 2 Links DO seem to be causing an invocation of the callback 
        dcc.Link('Navigate to "/plotary/dash/"', href='/plotary/dash/'),
        html.Br(),
        dcc.Link('Navigate to "/plotary/"', href='/plotary/'),
            
    ],
    id="navbar")]
)

@app.callback(Output('description', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    print(f'entering callback for pathname {pathname}')
    return html.Div([
        html.H1('Auswertung Nr {}'.format(pathname))
    ])

# if __name__ == '__main__':
#     app.run_server(port=8882,debug=False)

In [ ]:
import numpy as np
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import dash_table

def make_dt(table_id,df,display_headers=True):
    dt = dash_table.DataTable(
        id=table_id,
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="custom",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
#         style_cell={
#             'width': '{}%'.format(len(df.columns)),
#             'textOverflow': 'ellipsis',
#             'overflow': 'hidden'
#         }
    )
    if not display_headers:
        dt.style_header = {'display': 'none'}        
    return dt

    
def make_text_centered_div(text):    
    col_inner_style = {
        'margin':'auto',
        'word-break':'break-all',
        'word-wrap': 'break-word',
        'text-align':'center'
    }
    return html.Div([text],style=col_inner_style)

# make simple DataFrame
x = np.arange(0,20)
y = x**2
df = pd.DataFrame({'x':x,'y':y})

# create an "aggregates" DataFrame
df_agg = pd.DataFrame({'x':[int(df.x.mean())],'y':[df.y.sum()]})

# create dash_tables
dt = make_dt('mydt',df)
dt_agg = make_dt('mydt_agg',df_agg,display_headers=False)
agg_title = make_text_centered_div("aggregates")

# make app
app = dash.Dash(__name__)
app.layout = html.Div([dt,agg_title,dt_agg])
app.title = "My DataFrame With Aggs"

@app.callback(
    Output('mydt_agg','data'),
    [Input('mydt', 'rows'),
     Input('mydt', 'selected_row_indices')])
def selected_callback(rows,selected_row_indices):
    try:
        df_selected_rows=pd.DataFrame(rows)
        df_new_agg = pd.DataFrame({'x':[df_selected_rows.x.mean()],'y':[df_selected_rows.y.sum()]})
        d = df_new_agg.to_dict('records')
        return d
    except Exception as e:
        print(str(e))
        print(rows,selected_row_indices)
        raise PreventUpdate()

@app.callback(
    Output('mydt','data'),
#     [Input('mydt','filter_query')]
    [Input('mydt','derived_filter_query_structure')]
)
def process_filter(filter_query):
    print(filter_query,type(filter_query))
    return df.to_dict('records')
    if filter_query is None or len(filter_query)<=0:
        return df.to_dict('records')
    df_temp = df.copy()
    #{x} > 10 && {y} > 50
    fq = str(filter_query).replace('"','').replace('{',"").replace('}',"").replace('&&','and')
    print(fq)
    df_temp = df_temp.query(fq)
    return df_temp.to_dict('records')


app.run_server(host='127.0.0.1',port=8884)


In [ ]:
import io
import pathlib
h = pathlib.Path.home()
nym = open(f'{h}/downloads/nymex.settle.20200619.s.csv','r').readlines()
sio = io.StringIO()
sio.writelines(nym)
sio.seek(0)
df = pd.read_csv(sio)


In [ ]:
len(df),len(nym)

In [ ]:
import zipfile
z= f'{h}/downloads/nymex.settle.20200619.s.csv.zip'
f = zipfile.ZipFile(z).open('nymex.settle.20200619.s.csv')
nym2 = [l.decode("utf-8")  for l in f]
sio2 = io.StringIO()
sio2.writelines(nym2)
sio2.seek(0)
df2 = pd.read_csv(sio2)


In [ ]:
df2

In [ ]:
import zipfile
# z= f'{h}/downloads/nymex.settle.20200619.s.csv.zip'
z= f'{h}/downloads/df_avg_implied.csv.zip'
zo = open(z,'rb').read()
zoio = io.BytesIO()
zoio.write(zo)
# f = zipfile.ZipFile(zoio).open('nymex.settle.20200619.s.csv')
f = zipfile.ZipFile(zoio).open('df_avg_implied.csv')
nym2 = [l.decode("utf-8")  for l in f]
sio2 = io.StringIO()
sio2.writelines(nym2)
sio2.seek(0)
df2 = pd.read_csv(sio2)


In [ ]:
zo

In [ ]:
zoio.read()

In [ ]:
df2

In [ ]:
import re
re.findall("\.zip$",'df_avg_implied.csv.zip'.lower())

In [ ]:
import base64
c = "UEsDBBQACAAIAOl49lAAAAAAAAAAAAAAAAASABAAZGZfYXZnX2ltcGxpZWQuY3N2VVgMAGaOGF9mjhhf9QEUAG2aSa/jupKE9/wtFw8iKVHSUp6OXR7L1vHxqU2jV73ph7do4AH97/uLTMoF9K3CHVBhmyJziIxM6n/+959//ee//+s//v2v/w7TdPor/qNph9y2pUlN1/RDbMNlfdr/1fxjLG3uuiF1cexyTF24r0+Cc2qavovdUMbYNn3YbYXGcUxd7mMz5r4L020S2LRp7Ad+wO/bcQxrW2EYWbfrl3VjWF/nb+F91zR60OdkXytjKm3TpDJ0XVv6cHw8BDdjM+Sx1dqFn4SPm+B+BOtzn5vW0NNK4MB3YuzafmDDJYf5dhdc+mZMsU+G8sDz/WpwWzDBGBtHN6+10E5G4eta9fxxNoitd6Xp/FGfOmvf5lhKyc3QyQIR1OBUmr6M3dDItqmE3XwwuOmbboy9H2HzvAjkqHnk1HlIg+zw9X0RXsae4zY55XYoOY1hOh0FD33fsn6OQ+aZY9iY2cqQY8P39NSsJ96e8mbBDy3rZMzBQl1YaXulsIHYsOGxLzLQbrp8GI4NCIChzXJKCpvDw1bpGmwxYlYzUhvOe9tKm7ohltGt1MRwuT4Nbkpye4TnYZI1MVBm1yPHGfmwhO3rpvgpBEpp+eOWSuFxs30QAbj/t0+2886WiZnQi0NJ/cgvw+nbnhdj27XN0HAkfB7D42SLNF1HbLC8xcwY9l9nh2W+Gty76wvM9qpdc2j+xDCdN4KHthuHYWhKNd5+dTBYoRW77BFewvn6MLjp+qY0TWdnL2H+mhV2HINvkxBubXZ9WRs8jEPG+ZkMIkjC3vzY9ZG/sBPzDIsTjtp2R4CM5OuQeGQTcdnJHorfS7aU1O5jOH0+De5jmzlKOyiocjg9/dtkYIlNYqkxpbA3qxIAKRMoBLeiNWO/lcFNxtgZm3uQvO5mKxZg+SGlVMw3H1tDu4LH6tnJg6efnVwZa8KbYdfnyWD2gGUbZ46B0P5QyHetUiY34+h2DJevx+Q4th5jTbMS7maqllSIMFhN6bAzEEIYhzchnKe7o6QS2405c1CM/ZgNJoqqG0VC4Xq0w2Ssxicxml1z+NqsHe4baIidjKUZ27DZmVHTiOPbNg/VTvPL1k49iWuQr727VzgWGDONqR2atg/r6W47TCWmHsIiXjFXGz5Psx0deiUheXYiwpsxnG+2l5T5ZsY1sfQK7t365XDbVseYvW/Tt3+dtZuB1YkdzAUfXXw3HA+SLV0kAlI43D5uDpMlAxUi9UTXEE7Hn4KhukZc5QQRHt9mcaXo6MjudLg7lEnfRkGBAduwO8wWUhGqzemd1tPK80xpzfHSkNqekAuH5y+DE/aOVgbkysfh6YuQIW0pfe4oCANk9H2qMGGcnUOPB0ua2MA6TWVhonK/mizOmhFzmIe06/3W/NgMLVZi3axlc7icHv7lIRM6sF/r1WB/t21gCZwz8hOtUsJldfoTvL7u/g7D/XejEswMB/TwX0vJCp973wkWieP76GHefBjMXygJMdvuWcKCFUqNMdUKvH5dHVOUtxEC6wnu8PREJ1XaqERqFGUpPC/+7WaE0hZzREWBw0o7BQCU27dhf9CWVQcoBFn1mMRJrYrE+uifNGloxa7m3fXmy9CeP7CcuQKCDMftt+GlI59KVwt6sXUm+0QchfcrSedw3xmcolOlIufw4ZAKKlvxJKZ2bk8n+0D2GC0wLazC6qjAhA47nNl0C/HcXpu/w30fLnbUAbru86hzyTRhe7WnQkJIiY6ylSnBbThtHeb7xA2JLHt1pPDl/Ed8us1P+yD2ZAhGcRsTQs/N5B/wOBllNLcQAJPSkgLat6QW0sKW6slAM2XPqm0afPMYPBxmg1kCcgJuZTwkiwUduPynCLH44LRfZrSehMLiNdJh5fOvi+PUP1FzSxkbdeC7sQ2e7mBbkauH2eVqHidXCAN+NI4EZlSRnRxPaEsKqgUUfr3UcxV9QqTFJNoeg8kt0IFK1fTGzmG6bGdHOWCDfZoojRb2HjOlQNYDpak3+iPvDheLGpREYSP9wrfTeW2RUGDz1Hl9ZPmf9+fVYYphykgPJVbThut57XhU5rWewENY78znJSFIVN0t5ln+6350PJLwEgoW3Tz2YC4v0LVCzfZOqd2bDCL9qd0QDT5RHW/D/WoFCruqoo6LrAiHy/rb8Ux24xWTXymcX0Z4badnYn6XgZnCunI4oVLf5SXsT746jCAXlqqJISs7VddCnCKLyuoroyV+SNJSLBr9hkS+nK5Hx4k6KsOodFD2rOeT4/FdhmffB0uqoDUiZra9PvvzIhTURVyB6BxTuO8tTtld46JOa8B4X3tzHQEgWmgXHX3aXhzmFPzc6wKwBWOr3IMOKIFYn+bHIo5HwjxowWiHQS4/LKI5L1JZCRtVuSCxq3+fPZNJ5ECtdPcfFl1knToMMSU/CLf92VESshslP5NJgsdlchynoRYQcsoxStrX0XFFCRlPaCSY9u6EgnORsKT8wu6/VndfR8FJ/MKaKIAeZTZZVCgZf0uf/e2ngziSyvNue778u4M1EUs3ksPNIxoKZJkIZbcmcr6sdHB+6Zx+EX2PrcUt/o5aB4ksc4Xb9+3kuH5MwNEANrDvep4sglBECBCsNSxxsd6bfSm55S3PJLkwgTmVAkPThoSPvefv83R1nG9SVqLJJUSXBbSc8G7GwpfnEH/HnZF0Y1vtSIquHMYDEGnpa01ZHS9ugcZoNMEEqstduGzNq2oiqlYMazs+1Y5ggb5pQRAe9CMrhzO1eum52mCaiJggdGpwWjFZudPEexBdTc0UVl+Tw6gW6+pcA9xWaztNIgHtHC5GnpMvzkZZZBEvbfh1uJr9YNwkLlps8phfDsP/eMYrG3S2uvuJ8kBgVYKm4ptBpIXRHbXu0Ei5MxESmPAdVvP8ZZ6J2jP9L64hjtB+Xuw4I+Txu4MPz7tvJfY6PJFhao0qSFLY+qhNmlza1MaUCeVid/JfgEcFClWzSDmsXhZEhDnMnxHjbpuN00hMkuGlaiF2ert/Ok7QYvUhGnGFjVcvipD6avSo+YlOy5waY6FhQT1JqUmHf1YUmUIiWl9LDDkV0YqTilV9xAQNvyquvEJ4uIaWYDan0lFgn1iJlYZjsmdqldi+ZyYUHUfd8k5+s4nU/weu56cFkR6H16yvL1JS0+GX40idwv6ogpB/DsezWZYikwZKB/2dxARS5Tz5BokdBGxUGpHpfGCaGRyj1tw0Fnl4IuLS7GTuG1odrg7DWFBW21vian3zHFKzownpanMRdjsrlya9NKZaJOVq9e0wpTvJyhaqtACP9cE/IJdI4FJT/Xw6O9xpBZYxLu3D8WRjCT7Ians1OVEKhfWlwop4/uvc0IbL59ZxJCIio7EcJsueZ/8+8hiOGTwvE2XAUCKUJGmXPjLMu5fhRZVbDKPlaRFOhuJmnrnMlwoxMDuOUqcdreMDHL5xG4jX2DQrY+kh+IQEOKkUFVMvRNJ1PjwMJ6Jx7FIGwvzD90Ib2SzxhcQjNz6u/kGUsqM3NMETDqfzxXB4iZBfeAmRu7kbzplqX1YFz/x0l6CBxI9j7Y/D9uEbMt1OkenURmDj1/fpj/jnQ7mglOlMmCiSpXEnS3rICoWOjBlorFux9e1yd7yBHdpUBxVE4MHX6SllNFwu3zHE9uQwPEPDyhNMUE2bx9HwQgLqqBpJtUPYPu8fhncieMkwDec47/5xWPsHLNDgcXcB+zydLHg4vqkeRLy7gD7ILEEw0brCzmrN4xA+bmZpMYcTMY0VgR0Om9fLP2CfveYZ1iKhCizc4NfBSoMX1zA9vx3mr3h3gJF5LIXky+zDNpDdy+Qthtfp8nI8UROxTHFluZ5fD8cbTT8XNkIvHM2/FuQcKXmXET5Mdqk4j+pMlw50Ok6WimhQcsV8IL9I18/+AXTEQxUsKlZt2Po6iHeioHJ0G2aPEvQV4Zlqb9rxVEPJrYF1XJCX8MNBxDbKolhzQcRuZvMHmpcSK47NeB3R6sdJmr1RcXzIE7xIIYoII/LeBT06fLczmBiTQ51XxvBzffXDNO0gteA2R1c7iJ5YIowyettOFmJ8D33aR880TYT2DuPd9xwcPbvZ/BH+OltcI3F6Y4LOekCY1U5Je8naqW1dF1JxnLKwEkG31C0IfWMVA7+MGCz3te8iDw5/gh8rJ1DNHrET5cGkWDj51rOUKzVhlPPhQ6fVorHSMC5zmvDDTU7nxva6xQCaFj2e/oHqgYjWOlLY+eawGv6eVtWUR7if3TCNTSZIIQUQzr/Y1J+o4YBLs45pwrSzYKERJbKWEk0AzOfPh39AolOQ62A7bH/6OpA2xtdkv5X7jo9Ph9EDFM5sZQoeuJpIyZpea3ywNFGbjwpr0KZYt+3n8OO4tzxFuRQ6o2H0BOvCdVP3yXfZYF8jOIetMzHErRBo6iOUSc6skIMmHzCx0THU574CVxZl0y82LJptui+xjl+RMsUDc/9pQawpJdRB0e91z4NWt8rQJZtLDGrO2GgJ56PnCOaRemiHbFcqND033w/uTu9ZZU/Xf3ec3XRvotRCj4PjRAhJOCQTLuGw9eVpx7JKyVAkq8Pj++iwmoqxjiAifGtTwqzxlUblHuTh4+urolHalEpirRwN9WxubHW7odlW1IytC8f92uE+a34/GqmOqC03AkyHGfsqrCGryVH8tLTHyJXLwxehNrG1oaq+sLHM1GAyicQ0FaWy7DZ2TE1ISQ/sNYhboVmrN+rs+9+dPYnm3+aRRcMaz0xqhz+RfOriMnQaw2ljVxQK0VzbEovJk03pdQ8YoW9KnV9F3H2DmjWJJPEoEidMLmtoBUdNpUYvxmH98J0QocMoprVhCPF7P337B0VXIDCllTS8vJsdJzOxU6yXOYebo+QdujxbR0l+nLf12wgpWuo6lICv7K4p0wzQ2A12D2iN4PNuglgXRVK39YKFqno/L7gO1NV7T9jW12+SOGsYvAEN96slnwYJzuG6yRyzhpSOk1mQxjJVejnvZ7WqDWey31Bsty75TL0ogokhrBM+fBE8qsrfGx3y0Gm1dhwVYfMKa0TJpPX+yz+ILYdt31Pk9fXqOEXeYtXa4XBxjaJa8Z7KSDLdbV4D3ipfNKRX9Ibt4VXhXAdKVlOnzWrrOFxDVySBIY6/20hEMMd8X+pBD3VxojH1Q61E4TxvKkwBovnvk8nhsN5c3AhdI+PU+TrduaWjSKrvlnlYWHmM8TQ11GNfq9B5evwJ/rheXQ9gJA7TqDaOajD4geUpDy2k9cKFJMLtl+FKF0l+Ky4p/Li5V23+gDVLsthY3w1Og91VaaSrGhJO1j8iBumq6eabaFOR8HWaK45Ddbvn/Yzt0nHikPwlyWhq6MQNRAi1JvR8sLK1K3Tgxv5x0tTdx8VmHJkgH37vEMs8HNVA++1Tujwj2CSSqi2eRdHt7oskm4otFyNhO1vGJOn6uNxIdtRDjxdwdIhCQHmZggvpREnthlpj+F+YnHr+hh+qFQldTJ79doVgn6abL0Q3SyJBWEiIRI96uvguGzGM7spsGBPO17+jMOG0sSjF8VKry3Afwn94ckBWGmhI1A5RKbw93x1HWrLVxqZprPP0hojGF6qGO7xF64OjvarVMsGHaJ3ziPVOs2+8oisLMnj1aTbDE81bivYtseQu0XsVaOMOZ9Hxs835Y3YcdaQrOGvwqf0v32SnUF/uk0imp6+OKB6S7oRd6XwcTpZ8UWM+Yn5w6ifGrpbZCNpO8t01DbG3vV8cp3RKHDupoiVtMxrEsQBF3MQULvxpHNxomC8RaSHe6NLVYKpm8248kCHGD8qJvuuWEd3Kv0pJJ4uwevS59erucJKLiw8jkWJrV4watw7d+z5YLZbDVO9OQt2aUCSCK33OgRtIVrvh7lXzjWcJXYpPquM7dvPtwkcz2HrrLdYnZjyDKeyqvwSy5q+DvcWw8Q8gn9TUcW8Kl6uvnzLCpJDuUe/UhFXdpqbakI930IVT2eV8RqhnsWy9hKLe+uKSMJrikMbUW43wfjlOGYPeawenwucGVtvVqYeDJRDc4WbXeeBw2/B71j/fja+SmktYvF7opfBpKjtJCrkWspnZr/nhaAcvv+/mws3u55NGJRrIdnphZSSo1xXHP5Lli1Sf70+De3k6665f3XnY+trQAovQsdjkLfxY7ec/4buDf70Mw/KKkfXHp8Ol4nZDr7mUhgXBd4K44d+yiPrH3g/fqYhJ65ssQsyYvku6StBdaL1/4kRbhzXWjrn6qISN3UgQEFklb+ExWGO18pM2klUl1Zs5SS77AaE+DBqpRt2806o+bh+HP36wOn87rvbrnTktfcbF8aRXCnJjLoElJxNv1KbS6q2OzmiPA9sVMnC2qXwzmhiHIjYOq8zmOlDVzPO+NpwyU+Amn8ChZ6ZfZh66tU7dR70oCPPLNwOCxlGeSLp+mv5PigoIopTlGmP78sU7fcTzfEqo+9K94xqmK/p0QUV67O/mKytCwPWOTyMXN40mIuUti8J82BnMouX9EhihunUL6Ipa9bkOFI7uD83ax2hjfBsRWw1CmTYd2VFHV4SH9XaqMo00AphELZ2jWUVEsrxioRibNmeH8VdbX++CocNq46t0HK/t6uU/iTr9sB3qLq2VFnJBE272qh1wp6aA/0XT9OfN7LBWjjjIWA9JcPan6uKFUMFg4l+sZZ5WYerVdqHVjavXC06HXAu8r6sbYqJtaRf5ojULIkxq8vucSOWHb1v6o6vvFojDT2dfSFMACsxgerxwHitKSYMDGoNSrz278HQUHUOgRH9tSy+wvBzX2KCtl2FtuF0sJoq9naWbWRsEhctm+um4pB/OLOZRaRnbPMGvwcly2xIeni1F16nZLSXSD7cP/3qnm3VdINgtWZi2/vVWg39W8eugcKmw5tV6t86cMYTLscJiHhq7tg6apg+HdT0K/djNLjrRzVU0k6Cu51In7WvTchpLURsRso1u7EjRyQlAV97q4NvWbiGIrskN1ugqB7K32xOWuUlY6p4o/74nyixztos2VIrG3ZKMlZSn1ex4ryv7vr62g6C3G25UhgayhF523T7N1woTvJLcpb4zZ+9qSZRo3FpfVAu3td1bJSU4MMFmoyJK886XgZOX10r1GlswJQ7VkQAkY/KmQyXVN5PUHtolpV1nGajSrUGET4050FeFSThdw/qbrKgYu8yjmcvJ3m9YrjKu/uYrWYT6GJr33G43meaWFhr0o3q9Qqu6tsAmGjUCzvVVrhQeP2zz2cZciSDws543p1+GD4MW9gEznjofjR7xnd6AGuutq95tcVy1He2QfdbeoerNs1nzNipkA71bPr1MkMsgSRWo1kM8vr44HvVmFCpNs5OwcQxDsUvjBr0fs3VYIdTryTb70sxRMLHq9dC9SjaZ73DqWKPX3g/0co2D9Jpc5+87SoWtbBECOqsxQlhotkqg2q6j3nRpuuWGEnLcmiyJ6pL69/sZtHQ2zwTXm8yli8tY9Gh7wU58dViGXm3YfFoQ6L0ComV5WxPykStgRaIjv9/VgfD9PQxOqKYcLWs1aaCY3AxWqEiJ9T4VDD8umjkrupRf1GAzZDAFZu9r8ARiyV9dub5Uv3S+9wXX/wFQSwcI8AwO7zAWAACoLQAAUEsDBAoAAAAAAHJ8+1AAAAAAAAAAAAAAAAAJABAAX19NQUNPU1gvVVgMAIcsH1+HLB9f9QEUAFBLAwQUAAgACADpePZQAAAAAAAAAAAAAAAAHQAQAF9fTUFDT1NYLy5fZGZfYXZnX2ltcGxpZWQuY3N2VVgMAGaOGF9mjhhf9QEUAGNgFWNnYGJg8E1MVvAPVohQgAKQGAMnEBsBcRcQg/h7GIgCjiEhQVAmSMcMIFZBU8KIEBdNzs/VSywoyEnVKyxNLErMK8nMS2Uo1DcwsDCyNk0ztLBINTe39s1MLsovzk8riakwMnCtSE7NsWYAAFBLBwhdNRAjbwAAALwAAABQSwECFQMUAAgACADpePZQ8AwO7zAWAACoLQAAEgAMAAAAAAAAAABApIEAAAAAZGZfYXZnX2ltcGxpZWQuY3N2VVgIAGaOGF9mjhhfUEsBAhUDCgAAAAAAcnz7UAAAAAAAAAAAAAAAAAkADAAAAAAAAAAAQP1BgBYAAF9fTUFDT1NYL1VYCACHLB9fhywfX1BLAQIVAxQACAAIAOl49lBdNRAjbwAAALwAAAAdAAwAAAAAAAAAAECkgbcWAABfX01BQ09TWC8uX2RmX2F2Z19pbXBsaWVkLmNzdlVYCABmjhhfZo4YX1BLBQYAAAAAAwADAOYAAACBFwAAAAA="
content_decoded = base64.b64decode(c)
# Use BytesIO to handle the decoded content
zoio2 = io.BytesIO(content_decoded)
f = zipfile.ZipFile(zoio2).open('df_avg_implied.csv')
nym2 = [l.decode("utf-8")  for l in f]
sio2 = io.StringIO()
sio2.writelines(nym2)
sio2.seek(0)
df = pd.read_csv(sio2)



In [ ]:
df

In [ ]:
app = dash.Dash()
inp = dcc.Input(id='myinput',)
app.layout = html.Div([inp,html.Div(id='c1'),html.Div(id='c2')])

@app.callback(
    [Output('c1', 'children'),
     Output('c2', 'children')],
    [Input('myinput', 'value')])
def update_graph(v):
    return v,v.upper()
app.run_server(host='127.0.0.1',port=8844)

In [ ]:
from dashapp import dashapp2 as dashapp
import importlib
importlib.reload(dashapp)
inp = dcc.Input(id='myinput')
dap = dashapp.DashApp()
layout = html.Div([inp,html.Div(id='c1'),html.Div(id='c2')])
def _dd(input_data):
    return [input_data[0][::-1],input_data[0].upper()]
d = dashapp.DashLink([(inp,'value')],[('c1','children'),('c2','children')],_dd)
dap.add_links([d])
dap.create_app(layout,app_port=8844)


In [ ]:
?dashapp.make_dashtable

In [ ]:
from dashapp import dashapp2 as dashapp
import importlib
importlib.reload(dashapp)
inp = dcc.Input(id='myinput')
s = dcc.Store(id='mystore')
d1 = dashapp.DashLink(
    [(inp,'value')],
    [(s,'data')],
    lambda i:[pd.DataFrame({'d':[i[0],i[0][::-1]]})])

t,tlink = dashapp.make_dashtable('dtable',df_in=pd.DataFrame(),
                input_store=s,update_columns=True)


dap = dashapp.DashApp()
layout = html.Div([inp,html.Div(id='c1'),html.Div(id='c2'),t,s])

d2 = dashapp.DashLink([(s,'data')],[('c1','children')],
                     lambda i:[i[0]['d']])
d3 = dashapp.DashLink([(s,'data')],[('c2','children')],
                     lambda i:[i[-1]['d'][::-1]],
                     )
dap.add_links([d1,d2,d3,tlink])
dap.create_app(layout,app_port=8844)


In [ ]:
# use an dcc.Upload to get new zip file
import base64
import io
import zipfile
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import dash_table
import pandas as pd
import numpy as np



def zipdata_to_df(contents,filename=None):
    content_decoded = base64.b64decode(contents)
    # Use BytesIO to handle the decoded content
    zoio2 = io.BytesIO(content_decoded)
    return zipfile_to_df(zoio2,filename)

def zipfile_to_df(z,filename=None):
    z = zipfile.ZipFile(z)
    fn = filename
    if fn is None:
        fn = z.namelist()[0]
    f = z.open(fn.replace('.zip',''))
    nym2 = [l.decode("utf-8")  for l in f]
    sio2 = io.StringIO()
    sio2.writelines(nym2)
    sio2.seek(0)
    df = pd.read_csv(sio2)
    return df

def df_to_zipdata(df,filename):
    sio2 = io.StringIO()
    df.to_csv(sio2,index=False)
    sio2.seek(0)
    zoio2 = io.BytesIO()
    f = zipfile.ZipFile(zoio2,'a',zipfile.ZIP_DEFLATED,False)
    f.writestr(filename,sio2.read())
    f.close() 
    zoio2.seek(0)
    return zoio2


port = 8812
url_base_pathname='/app8812/'
app = dash.Dash(url_base_pathname=url_base_pathname)
main_id='myid'
upl = dcc.Upload(
    id=f"{main_id}_uploader",
    children=html.Div(['choose zip file']),
    accept = '.zip'
)


w = html.Div([],id=f"{main_id}_main_window")


def _make_dt(dt_id,df,displayed_rows=100,page_action='native'):
    dt = dash_table.DataTable(
        id=dt_id,
        page_current= 0,
        page_size=displayed_rows,
        page_action=page_action,        
        
    )
    dt.data=df.to_dict('rows')
    dt.columns=[{"name": i, "id": i} for i in df.columns.values]                    
    return dt

savec=None
@app.callback(
    [Output(w.id,'children')],
    [Input(upl.id,'contents')]
)
def update_window(rawzip):
    global savec
    if (rawzip is None) or (len(rawzip)<2):
        raise PreventUpdate()        
    c = rawzip.split(",")[1]
    savec = c
    df = zipdata_to_df(c)
    dt = _make_dt(f"{main_id}_dashtable",df,displayed_rows=1000)
    return [dt]

all_rows = [upl,dcc.Loading(children=[w])]
app.layout = html.Div(all_rows)
app.run_server(port=port)

In [ ]:
import datetime
n = datetime.datetime.now() - datetime.timedelta(1)
day_of_week = n.weekday()
days_to_subtract = day_of_week - 1
if days_to_subtract <0:
    days_to_subtract = 7+days_to_subtract
past_tuesday = n - datetime.timedelta(days_to_subtract)
y = past_tuesday.year
m = past_tuesday.month
d = past_tuesday.day
yyyymmdd = int(str(y))*100*100 + int(str(m))*100 + int(str(d))
yyyymmdd


In [ ]:
# Use zip base64 encoded string of a cme settlement file with over 100k rows
import base64
import io
import zipfile
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import urllib

import dash_table
import pandas as pd
import numpy as np
import datetime

def get_prev_tuesday_yyyymmdd():
    n = datetime.datetime.now() - datetime.timedelta(1)
    day_of_week = n.weekday()
    days_to_subtract = day_of_week - 1
    if days_to_subtract <0:
        days_to_subtract = 7+days_to_subtract
    past_tuesday = n - datetime.timedelta(days_to_subtract)
    y = past_tuesday.year
    m = past_tuesday.month
    d = past_tuesday.day
    yyyymmdd = int(str(y))*100*100 + int(str(m))*100 + int(str(d))
    return yyyymmdd

def zipdata_to_df(contents,filename=None):
    content_decoded = base64.b64decode(contents)
    # Use BytesIO to handle the decoded content
    zoio2 = io.BytesIO(content_decoded)
    return zipfile_to_df(zoio2,filename)

def zipfile_to_df(z,filename=None):
    z = zipfile.ZipFile(z)
    fn = filename
    if fn is None:
        fn = z.namelist()[0]
    f = z.open(fn.replace('.zip',''))
    nym2 = [l.decode("utf-8")  for l in f]
    sio2 = io.StringIO()
    sio2.writelines(nym2)
    sio2.seek(0)
    df = pd.read_csv(sio2)
    return df

def df_to_zipdata(df,filename):
    sio2 = io.StringIO()
    df.to_csv(sio2,index=False)
    sio2.seek(0)
    zoio2 = io.BytesIO()
    f = zipfile.ZipFile(zoio2,'a',zipfile.ZIP_DEFLATED,False)
    f.writestr(filename,sio2.read())
    f.close() 
    zoio2.seek(0)
    return zoio2

def get_cme_df(yyyymmdd,multiplier=1):
    url=f'ftp://ftp.cmegroup.com/settle/cme.settle.{yyyymmdd}.s.csv.zip'
    # get the zip file
    mysock = urllib.request.urlopen(url)
    memfile = io.BytesIO(mysock.read())
    df_main = zipfile_to_df(memfile)
    df_main = df_main.append(df_main)
    return df_main
    

def get_cme_zipfile_string(yyyymmdd,multiplier=1):
    df_main = get_cme_df(yyyymmdd,multiplier=multiplier)
    print(len(df_main))
    zd = df_to_zipdata(df_main,'df_main.csv')
    zdstring = base64.b64encode(zd.read()).decode("utf-8")
    return zdstring
    

# create a port for the Dash app
port = 8812
# I'm using a nginx proxy server, so I am routing requests from my public URL
#   to a specific route (/app8812/)
# For my domain, I would enter https://billybyte.com/app8812/ in my browser address bar
url_base_pathname='/app8812/'

# create a dash app
app = dash.Dash(url_base_pathname=url_base_pathname)

# use this prefix as the prefix to all component id's
main_id='myid'

# get a tuesday, which is probably a day where there was trading on the CME
yyyymmdd = get_prev_tuesday_yyyymmdd()
# get zip file that has about 100k rows of csv data from the CME Exchanges public FTP site
# zdstring will be a compressed version of a csv file, that will be less than one tenth
#    the actual size of the csv file.
zdstring = get_cme_zipfile_string(yyyymmdd,multiplier=2)
# create dcc.Store that holds the string of compressed csv data
zipstore = dcc.Store(id=f"{main_id}_zipstore",data=zdstring)

# create an input box into which the user can enter a query that conforms to the syntax
#   of pd.DataFrame.query()
inp = dcc.Input(
    id=f"{main_id}_input",debounce=True,
    placeholder="Enter Query like: Sym=='ES' & PrevDayVol>250",
    style = dict(width = '50%',display = 'table-cell')
)

# create a div that will receive the dash_table.DataTable component of the CME data
main_window = html.Div([],id=f"{main_id}_main_window")

# This method get's called in the callback, to create the dash_table.DataTable object
def _make_dt(dt_id,df,displayed_rows=100,page_action='native'):
    dt = dash_table.DataTable(
        id=dt_id,
        page_current= 0,
        page_size=displayed_rows,
        page_action=page_action,        
        
    )
    dt.data=df.to_dict('rows')
    dt.columns=[{"name": i, "id": i} for i in df.columns.values]                    
    return dt

# This callback get's called when:
#   1. When the webpage get's initially loaded, and displays all 100k+ rows of data, or
#   2. After you enter a query in the dcc.Input component and hit enter or tab
@app.callback(
    [Output(main_window.id,'children')],
    [Input(inp.id,'value'),Input(zipstore.id,'data')]
)
def update_window(query,zipfile):
    if (zipfile is None) or (len(zipfile)<1):
        raise PreventUpdate()        
    df = zipdata_to_df(zipfile)
    if (query is not None) and (len(query)>0):
        df = df.query(query)
    dt = _make_dt(f"{main_id}_dashtable",df)
    return [dt]

# create the layout div
all_rows = [inp,dcc.Loading(children=[main_window],fullscreen=True),zipstore]
app.layout = html.Div(all_rows)
# run the app
app.run_server(port=port)

In [ ]:
# use CallbackCache
from dash.dependencies import Output, Input
from flask_caching.backends import FileSystemCache
from dash_extensions.callback import CallbackCache, Trigger
import dash_table

def get_prev_tuesday_yyyymmdd():
    n = datetime.datetime.now() - datetime.timedelta(1)
    day_of_week = n.weekday()
    days_to_subtract = day_of_week - 1
    if days_to_subtract <0:
        days_to_subtract = 7+days_to_subtract
    past_tuesday = n - datetime.timedelta(days_to_subtract)
    y = past_tuesday.year
    m = past_tuesday.month
    d = past_tuesday.day
    yyyymmdd = int(str(y))*100*100 + int(str(m))*100 + int(str(d))
    return yyyymmdd

def get_cme_df(yyyymmdd):
    url=f'ftp://ftp.cmegroup.com/settle/cme.settle.{yyyymmdd}.s.csv.zip'
    # get the zip file
    mysock = urllib.request.urlopen(url)
    memfile = io.BytesIO(mysock.read())
    df_main = zipfile_to_df(memfile)
    df_main = df_main.append(df_main)
    return df_main

yyyymmdd = get_prev_tuesday_yyyymmdd()
df_cme = get_cme_df(yyyymmdd)

def _make_dt(dt_id,df,displayed_rows=100,page_action='native'):
    dt = dash_table.DataTable(
        id=dt_id,
        page_current= 0,
        page_size=displayed_rows,
        page_action=page_action,        
        
    )
    dt.data=df.to_dict('rows')
    dt.columns=[{"name": i, "id": i} for i in df.columns.values]                    
    return dt


# Create app.
# app = dash.Dash(prevent_initial_callbacks=True)
app = dash.Dash()
app.layout = html.Div([
#     html.Button("Query data", id="btn"), dcc.Dropdown(id="dd"), dcc.Graph(id="graph"),
    html.Button("Query data", id="btn"), dcc.Input(id="dd",debounce=True), 
    dcc.Loading(html.Div(id="graph"),fullscreen=True),
    dcc.Store(id="store")
])
# Create (server side) cache. Works with any flask caching backend.
cc = CallbackCache(cache=FileSystemCache(cache_dir="cache"))



@cc.cached_callback(Output("store", "data"), [Trigger("btn", "n_clicks")])  # Trigger is like Input, but excluded from args
def query_data():
    time.sleep(1)  # sleep to emulate a database call / a long calculation
#     return px.data.gapminder()
    return df_cme

search_col = 'Sym'

@cc.callback(Output("graph", "children"), [Input("store", "data"), Input("dd", "value")])
def update_graph(df, value):
    if (value is not None) and (len(value)>0):
        try:
            df = df.query(value)
        except Exception as e:
            print(str(e))
#     return px.sunburst(df, path=['continent', 'country'], values='pop', color='lifeExp', hover_data=['iso_alpha'])
#     fig =  px.sunburst(df, path=['continent', 'country'], values='pop', color='lifeExp', hover_data=['iso_alpha'])
#     g = dcc.Graph(figure=fig)
    dt = _make_dt(f"dtid",df)
    return dt


# This call registers the callbacks on the application.
cc.register(app)

if __name__ == '__main__':
    app.run_server()

In [ ]:
?dash_table.DataTable

In [ ]:
h = pathlib.Path.home()
csvname="nymex.settle.20200609.s.csv"
fullpath = f"{h}/downloads/{csvname}"
df = pd.read_csv(fullpath)
z = df_to_zipdata(df,csvname)
# df2 = zipfile_to_df(z,csvname)
df2 = zipfile_to_df(z)
df2.head(5)
# zoio2.seek(0)
# len(zoio2.read())

In [ ]:
lines = open(f"{h}/downloads/billionaires.txt").readlines()
ss = "[0-9]{1,3}[ ]([A-Z a-z]+)(\$[0-9]{1,4}[.][0-9]{1,2}B)"
pairs = [re.findall(ss,l) for l in lines]
vlist = [float(s[0][1].replace('B','').replace('$','')) for s in pairs if len(s)>0]
sum(vlist),len(vlist)

In [ ]:
pairs

In [ ]:
import dash_core_components as dcc
import dash_html_components as html
from dash_extensions.enrich import Dash, ServersideOutput, Output, Input, State, Trigger
from dash.exceptions import PreventUpdate
import dash_table

class ColumnSelector(dash_table.DataTable):
    def __init__(self,dt_id,options=None,
                 options_input_dashtable=None,
                 displayed_rows=4,
                 value=None,style=None):
        
        self.options_input_dashtable=options_input_dashtable
        opts = options
        if opts is None:
            opts = []
        df = pd.DataFrame({'option':opts})            
        data=df.to_dict('rows')
        columns=[{"name": i, "id": i} for i in df.columns.values]                    
        selected_rows=list(range(len(df)))
        
        super(ColumnSelector,self).__init__(
            id=dt_id,
            editable=True,
            page_action='none', 
            style_table={
                'overflowY':'auto',
                'height': f'{30*(displayed_rows+1)+2}px'
            } ,
            fixed_rows={'headers': True},
            row_selectable='multi',
            data=data,
            columns=columns,
            selected_rows=selected_rows
        )
    def register_app(self,theapp):
        if self.options_input_dashtable is not None:
            @theapp.callback(
                [
                    Output(self.id,'data'),
                    Output(self.id,'columns'),
                    Output(self.id,'selected_rows')],
                [Input(self.options_input_dashtable.id,'columns')]            
            )
            def _change_options(columns_dict):
                if columns_dict is None or len(columns_dict)<=0:
                    raise PreventUpdate("callback MultiDropdown._change_options: no DataFrame columns")
                names = [c['name'] for c in columns_dict]
                df_return = pd.DataFrame({'option':names})
                data = df_return.to_dict('records')
                columns = [{'name':c,'id':c} for c in df_return.columns.values]
                selected_rows=df_return.index.values
                return data,columns,selected_rows
            return _change_options
        else:
            return None
                
                

In [ ]:
opts = ['red','orange','yellow','green','blue','indigo','violet']
df_original= pd.DataFrame({c:list(range(10)) for c in opts})
dt = dash_table.DataTable(id='test_input_df',data=df_original.to_dict('records'),
                         columns=[{'name':c,'id':c} for c in df.columns.values])
dt2 = dash_table.DataTable(id='test_input_df2',data=df_original.to_dict('records'),
                         columns=[{'name':c,'id':c} for c in df.columns.values])

mdd = ColumnSelector('mdd_id',options_input_dashtable=dt)
but = html.Button("Click to change cols",id='change_cols')

def reg_but(theapp,original_columns):
    @theapp.callback(
        Output(dt2.id,'columns'),
        [Input(but.id,'n_clicks')],
        [State(mdd.id,'data'),State(mdd.id,'selected_rows')]        
    )
    def _change_cols(n_clicks,data,selected_rows):
        
        columns = [{'name':c,'id':c} for c in original_columns]
        if len(data)>0:
            df_mdd = pd.DataFrame(data).loc[selected_rows].sort_index()
            cols_to_show = df_mdd['option'].values
            columns = [{'name':c,'id':c} for c in cols_to_show]
        return columns
    return _change_cols   
        


In [ ]:
app = dash.Dash()
padd = 1

app.layout = html.Div([mdd,but,dt,dt2],
                      style={
                          'padding-right': f'{padd}%',
                          'padding-left': f'{padd}%',
                          'display':'grid',
                          'grid-template-columns':'1fr 1fr 4fr 4fr',
                          'grid-template-rows':'1fr'
                      }
                     )

reg_but(app,opts)
mdd.register_app(app)

if __name__ == '__main__':
    app.run_server(port=8813)

In [ ]:
df

In [ ]:
import pandas as pd
import dash
import dash_table
import dash_html_components as html


df = pd.DataFrame({'mycol':range(10)})
d = df.to_dict('records')
cols = [{'name':c,'id':c} for c in df.columns.values]
dt1 = dash_table.DataTable(
    id='dt1',
    data=d,
    columns=cols,
    page_current= 0,
    page_size=4,
    page_action='native'
)

dt2 = dash_table.DataTable(
    id='dt2',
    data=d,
    columns=cols,
    page_current= 0,
    page_size=4,
    page_action='native' 
)

app = dash.Dash()
s = {'display':'grid'}
app.layout=html.Div(
    [dt1,dt2],
    style=s
)
if __name__ == '__main__':
    app.run_server()

In [ ]:
!pip list|grep dash

In [ ]:
import pathlib
h = pathlib.Path.home()
from dashapp import single_page_from_df as spfd#@UnresolvedImport
df = pd.read_csv(f"{h}/downloads/data_csv.csv")



In [ ]:
import progressive_dropdown as progdd
import dash_core_components as dcc
import dash_html_components as html

init_values_source = dcc.Store(id='mystore',data=df.to_dict('records'))
pdd = progdd.ProgressiveDropdown(init_values_source,'pddd',3)


In [ ]:
df.dtypes[(df.dtypes=='object') | (df.dtypes=='float64')].index

In [ ]:
df.dtypes

In [ ]:
class Test1(html.Div):
    def __init__(self,test_id,myvalue=-1):
        self.myvalue = myvalue
        super(Test1,self).__init__(['testit'],id=f'test1{test_id}')

In [ ]:
t1 = Test1('t1',10)

In [ ]:
import urllib.request, json 

In [ ]:
with urllib.request.urlopen("https://yidi.imfast.io/c03db104479aa2d6f3c49743cb01d60ab190d8b3/data/Pinnacle_Premier%20League.json") as url:
    data = json.loads(url.read().decode())
    print(data)

In [ ]:
data['0']

In [ ]:
import pg_pandas as pg

In [ ]:
pga = pg.PgPandas(
    username='root',password='CJu84cs@cI9',databasename='thequantedge',
    dburl='127.0.0.1:3306',dbflavor='mysql+pymysql://')


In [ ]:
sql = """
select * from membership_orders;
"""
df_mo = pga.get_sql(sql)

In [ ]:
len(df_mo)

In [ ]:
sql = """
select * from users;
"""
df_users = pga.get_sql(sql)

In [ ]:
df_users

In [ ]:
df_spi_matches = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv")

In [ ]:
dff

In [ ]:
import pathlib
h = pathlib.Path.home()
d = f"{h}/downloads"
df_pop_by_state = pd.read_csv(f"{d}/population_by_state.csv")
df_electoral_college_by_state = pd.read_csv(f"{d}/electoral_college_by_state.csv")
df_pop_vote_by_state = pd.read_csv(f"{d}/popular_vote_2016_by_state.csv")
df_pop_vote_by_state.Votes = df_pop_vote_by_state.Votes.str.replace(',','').astype(int)

In [ ]:
df_pop_vote_by_state[df_pop_vote_by_state['State Code']=='AZ']

In [ ]:
print(df_electoral_college_by_state.columns.values)
print(df_pop_by_state.columns.values)
print(df_pop_vote_by_state.columns.values)

In [ ]:
df_electoral = df_electoral_college_by_state.merge(df_pop_by_state,on='State',how='inner')

In [ ]:
df_electoral['percapita_power'] = 1000000* df_electoral.qty/df_electoral['2018 Population']

In [ ]:
df_electoral.sort_values('percapita_power',ascending=False)

In [ ]:
df_pop_vote_by_state_pt = pd.pivot_table(
    df_pop_vote_by_state,index=['State'],
    values='Votes',columns=['Party'], aggfunc=np.sum)
df_pop_vote_by_state_pt = df_pop_vote_by_state_pt.fillna(0)#[['DEM','REP']]
df_pop_vote_by_state_pt.index.name = None
df_pop_vote_by_state_pt['State'] = df_pop_vote_by_state_pt.index
df_pop_vote_by_state_pt.index = list(range(len(df_pop_vote_by_state_pt)))
df_pop_vote_by_state_pt = df_pop_vote_by_state_pt[['State','DEM','REP']]
# df_pop_vote_by_state_pt.DEM = df_pop_vote_by_state_pt.DEM.str.replace(',','').astype(int)
# df_pop_vote_by_state_pt.REP = df_pop_vote_by_state_pt.REP.str.replace(',','').astype(int)

In [ ]:
df_pop_vote_by_state_pt

In [ ]:
df_electoral_2 = df_electoral.merge(df_pop_vote_by_state_pt,on='State')
df_electoral_2['dem_stranded'] = df_electoral_2.apply(
    lambda r: r.DEM if r.DEM < r.REP else 0,axis=1)
df_electoral_2['rep_stranded'] = df_electoral_2.apply(
    lambda r: r.REP if r.REP < r.DEM else 0,axis=1)


In [ ]:
df_electoral_2

In [ ]:
print(df_electoral_2.dem_stranded.sum(),df_electoral_2.DEM.sum())
print(df_electoral_2.rep_stranded.sum(),df_electoral_2.REP.sum())
num_dem_wins = len(df_electoral_2[df_electoral_2.rep_stranded>0])
num_rep_wins = len(df_electoral_2[df_electoral_2.dem_stranded>0])
print(num_dem_wins,num_rep_wins)
df_electoral_2['perc_pop_voted'] = (df_electoral_2.DEM + df_electoral_2.REP)/df_electoral_2['2018 Population']
df_electoral_2[['State','2018 Population','perc_pop_voted']]

In [ ]:
df_electoral_2[df_electoral_2.dem_stranded>0].sort_values('dem_stranded',ascending=False)

In [ ]:
sp_pe_monthly_txt = open(f"{d}/sp500_pe_monthly.txt",'r').readlines()


In [ ]:
import re
l = sp_pe_monthly_txt[3].replace('\n','').replace(',','').replace('\t',' ').lower()
re.split('[a-z]{1,5}[ ][0-9]{1,2}[ ]{1,3}[1-2][0-9]{3}',l)
a = [l.replace('\n','').replace(',','').replace('\t',' ').lower() for l in sp_pe_monthly_txt[1:]]
prices = [re.split('[a-z]{1,5}[ ][0-9]{1,2}[ ]{1,3}[1-2][0-9]{3}',l)[1].strip() for l in a]
months = {'jan':1,'feb':2,'mar':3,'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
dates = [re.split('[0-9]{1,3}[.][0-9]{2}',l)[0].strip().split(' ') for l in a]
dates = [int(dd[2])*100*100+months[dd[0]]*100+int(dd[1]) for dd in dates]
prices = [float(p) for p in prices]
df_pe = pd.DataFrame({'yyyymmdd':dates,'pe':prices})

In [ ]:
df_pe[(df_pe.yyyymmdd>20080101) & (df_pe.yyyymmdd<20100101)]

In [ ]:
df_pe[df_pe.yyyymmdd>20080101].plot(x='yyyymmdd',y='pe',kind='line')

In [ ]:
import urllib.request
response = urllib.request.urlopen('https://storage.googleapis.com/nflnfltest/public/nfl_lineup.json')
nfl_impact_text = response.read()
nfl_impact_dicts = json.loads(nfl_impact_text)


In [ ]:
nfl_impact_dicts[1]['lineup']

In [ ]:
[nfl['team_name'] for nfl in nfl_impact_dicts]

## Various examples with Subplots

### Basic subplots example

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import re

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/Mining-BTC-180.csv")

for i, row in enumerate(df["Date"]):
    p = re.compile(" 00:00:00")
    datetime = p.split(df["Date"][i])[0]
    df.iloc[i, 1] = datetime

fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)

fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["Mining-revenue-USD"],
        mode="lines",
        name="mining revenue"
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["Hash-rate"],
        mode="lines",
        name="hash-rate-TH/s"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Table(
        header=dict(
            values=["Date", "Number<br>Transactions", "Output<br>Volume (BTC)",
                    "Market<br>Price", "Hash<br>Rate", "Cost per<br>trans-USD",
                    "Mining<br>Revenue-USD", "Trasaction<br>fees-BTC"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns[1:]],
            align = "left")
    ),
    row=1, col=1
)
fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Bitcoin mining stats for 180 days",
)

fig.show()


### simpler version of the above

In [ ]:
fig2 = pu.plotly_plot(df[['Date','Number-transactions']],x_column='Date')
# fig2['data']
figm = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[
        [{"type": "table"}],
        [{"type": "scatter"}],
#         [{"type": "scatter"}]
    ]
)
tab1 =   go.Table(
        header=dict(
            values=["Date", "Number<br>Transactions", "Output<br>Volume (BTC)",
                    "Market<br>Price", "Hash<br>Rate", "Cost per<br>trans-USD",
                    "Mining<br>Revenue-USD", "Trasaction<br>fees-BTC"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns[1:]],
            align = "left")
)

figm.add_trace(tab1,row=1,col=1)
figm.add_trace(fig2['data'][0],row=2,col=1)
figm.update_layout(fig2['layout'])

# fig2['data'][0]

### Use Suplots with dash_bootstrap_components 
use dcc.Row and dcc.Col to form a simple grid, within a dcc.Tabs component

In [ ]:
"""
A simple app demonstrating how to dynamically render tab content containing
dcc.Graph components to ensure graphs get sized correctly. We also show how
dcc.Store can be used to cache the results of an expensive graph generation
process so that switching tabs is fast.
"""
import time

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import numpy as np
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots


app = dash.Dash(external_stylesheets=[dbc.themes.LUX])

app.layout = dbc.Container(
    [
        dcc.Store(id="store"),
        html.H1("Dynamically rendered tab content"),
        html.Hr(),
        dbc.Button(
            "Regenerate graphs",
            color="primary",
            block=True,
            id="button",
            className="mb-3",
        ),
        dbc.Tabs(
            [
                dbc.Tab(label="Scatter", tab_id="scatter"),
                dbc.Tab(label="Histograms", tab_id="histogram"),
            ],
            id="tabs",
            active_tab="scatter",
        ),
        html.Div(id="tab-content", className="p-4"),
    ]
)


@app.callback(
    Output("tab-content", "children"),
    [Input("tabs", "active_tab"), Input("store", "data")],
)
def render_tab_content(active_tab, data):
    """
    This callback takes the 'active_tab' property as input, as well as the
    stored graphs, and renders the tab content depending on what the value of
    'active_tab' is.
    """
    if active_tab and data is not None:
        if active_tab == "scatter":
            return dcc.Graph(figure=data["scatter"])
        elif active_tab == "histogram":
            return dbc.Row(
                [
#                     dbc.Col(dcc.Graph(figure=data["hist_1"]), width=6),
#                     dbc.Col(dcc.Graph(figure=data["hist_2"]), width=6),
                    dbc.Col(dcc.Graph(figure=data['figm'])),
                ]
            )
        
    return "No tab selected"


@app.callback(Output("store", "data"), [Input("button", "n_clicks")])
def generate_graphs(n):
    """
    This callback generates three simple graphs from random data.
    """
    if not n:
        # generate empty graphs when app loads
        return {k: go.Figure(data=[]) for k in ["scatter", "hist_1", "hist_2"]}

    # simulate expensive graph generation process
#     time.sleep(2)

    # generate 100 multivariate normal samples
    data = np.random.multivariate_normal([0, 0], [[1, 0.5], [0.5, 1]], 100)

    scatter = go.Figure(
        data=[go.Scatter(x=data[:, 0], y=data[:, 1], mode="markers")]
    )
    figm = make_subplots(
        rows=1, cols=2,
        shared_xaxes=False,
        horizontal_spacing=0.03,
        specs=[
            [{"type": "scatter"},{"type": "scatter"}],
        ]
    )
#     figm.add_trace(go.Histogram(x=data[:, 0]),row=1,col=1)
#     figm.add_trace(go.Histogram(x=data[:, 1]),row=1,col=2)
    figm.add_traces(
        [
            go.Histogram(x=data[:, 0]),
            go.Histogram(x=data[:, 1])
        ],
        rows = [1,1],
        cols = [1,2]
    )
    
#     hist_1 = go.Figure(
#         data=[go.Histogram(x=data[:, 0])],
#         layout=go.Layout(margin={'t': 0})
#     )
#     hist_2 = go.Figure(
#         data=[go.Histogram(x=data[:, 1])],
#         layout=go.Layout(margin={'t': 0})
#     )

    # save figures in a dictionary for sending to the dcc.Store
#     return {"scatter": scatter, "hist_1": hist_1, "hist_2": hist_2, "figm":figm}
    return {"scatter": scatter, "figm":figm}


if __name__ == "__main__":
    app.run_server(debug=False, port=8884)

In [ ]:
data = np.random.multivariate_normal([0, 0], [[1, 0.5], [0.5, 1]], 100)

figm = make_subplots(
    rows=1, cols=2,
    shared_xaxes=False,
    horizontal_spacing=0.03,
    specs=[
        [{"type": "scatter"},{"type": "scatter"}],
    ]
)
figm.add_traces(
    [
        go.Histogram(x=data[:, 0]),
        go.Histogram(x=data[:, 1])
    ],
    rows = [1,1],
    cols = [1,2]
)

